Imports, Parameters, and Object Points

In [57]:
import cv2
import numpy as np
import glob

# Setting the expected chessboard pattern size (number of inner corners)
pattern_size = (9, 6)
square_size = 21.7  # length of each square in mm

# Preparing the object points (3D points in the chessboard coordinate system)
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size

# Lists to store all object points and image points
object_points_all = []   # 3D points in real-world space
image_points_all = []    # 2D points in the image plane

# For images that were processed automatically
object_points_auto = []
image_points_auto = []

training_files = glob.glob('training_images/*.jpg')

Helper Functions for corner detection

In [58]:
# Function that allows automatic inner corner detection
def detect_corners_automatically(gray_img, pattern_size):
    try:
        ret, corners = cv2.findChessboardCornersSB(gray_img, pattern_size, None)
    except Exception:
        # Fall back to findChessboardCorners with flags if findChessboardCornersSB is unavailable.
        flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
        ret, corners = cv2.findChessboardCorners(gray_img, pattern_size, flags)
    return ret, corners

# Manual inner corner detection function that is called if automatic corner detection fails
def get_manual_corners(img):
    clicked_points = []
    img_copy = img.copy()

    # Function that allows user to select the four outer corners of the chessboard (modified from geeksforgeeks.org)
    def click_event(event, x, y, _flags, _params):
        nonlocal clicked_points, img_copy
        if event == cv2.EVENT_LBUTTONDOWN:
            clicked_points.append((x, y))
            cv2.circle(img_copy, (x, y), 5, (0, 0, 255), -1)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img_copy, f"{x},{y}", (x, y), font, 0.5, (0, 0, 255), 1)
            print(f"Point selected: ({x}, {y})")
            cv2.imshow("Manual Annotation", img_copy)

    cv2.namedWindow("Manual Annotation", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Manual Annotation", 1280, 720)
    cv2.imshow("Manual Annotation", img_copy)
    cv2.setMouseCallback("Manual Annotation", click_event)

    print("Please click on the 4 corners of the chessboard in the following order:")
    print("1. Top-Left")
    print("2. Top-Right")
    print("3. Bottom-Right")
    print("4. Bottom-Left")

    while True:
        cv2.imshow("Manual Annotation", img_copy)
        key = cv2.waitKey(1) & 0xFF
        if len(clicked_points) == 4:
            print("4 points have been selected.")
            break
        if key == 27:
            print("Manual annotation canceled. Not enough points selected.")
            break
    cv2.destroyWindow("Manual Annotation")
    
    if len(clicked_points) == 4:
        return clicked_points
    else:
        return None

# Function that linearly interpolates all chessboard points from the given outer corners
def interpolate_with_homography(corners, grid_size):
    num_cols, num_rows = grid_size

    # Define the ideal source points in a rectified coordinate system for the inner corners
    src_points = np.array([
        [0, 0],                     # top-left of inner corners
        [num_cols - 1, 0],          # top-right of inner corners
        [num_cols - 1, num_rows - 1],  # bottom-right
        [0, num_rows - 1]           # bottom-left
    ], dtype=np.float32)
    dst_points = np.array(corners, dtype=np.float32)
    H = cv2.getPerspectiveTransform(src_points, dst_points)

    # Generate the full grid of points using a list comprehension and convert to a NumPy array
    grid_points = np.array([[j, i] for i in range(num_rows) for j in range(num_cols)], dtype=np.float32)
    
    # Reshape grid_points to (num_rows, num_cols, 2)
    grid_points = grid_points.reshape(num_rows, num_cols, 2)
            
    # Transform the full grid to image points using the homography
    full_points = cv2.perspectiveTransform(grid_points.reshape(-1, 1, 2), H)
    full_points = full_points.reshape(num_rows, num_cols, 2)
    
    # Extract only the inner points (exclude the outer rows and columns)
    inner_points = full_points[1:-1, 1:-1, :]
    return inner_points.reshape(-1, 2)

# Function that determines whether the chessboard is placed vertically or horizontally
def determine_grid_size(corners, horizontal_grid_size=11, vertical_grid_size=8):
    tl = np.array(corners[0], dtype=np.float32)
    tr = np.array(corners[1], dtype=np.float32)
    bl = np.array(corners[3], dtype=np.float32)
    width = np.linalg.norm(tr - tl)
    height = np.linalg.norm(bl - tl)
    if width >= height:
        return (horizontal_grid_size, vertical_grid_size)
    else:
        return (vertical_grid_size, horizontal_grid_size)

Process Each Training Image

In [59]:
for idx, fname in enumerate(training_files):
    print(f"\nProcessing image: {fname}")
    img_train = cv2.imread(fname)
    if img_train is None:
        print("Failed to load image.")
        continue
    gray_train = cv2.cvtColor(img_train, cv2.COLOR_BGR2GRAY)
    
    # Attempt automatic corner detection
    ret, corners = detect_corners_automatically(gray_train, pattern_size)
    
    if ret:
        print("Automatic corner detection succeeded.")
        # Refine corner positions for better accuracy
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners_refined = cv2.cornerSubPix(gray_train, corners, (11, 11), (-1, -1), criteria)

        # Add the refined corners and corresponding object points
        object_points_all.append(objp)
        image_points_all.append(corners_refined)
        
        # Also store these in the automatic lists
        object_points_auto.append(objp)
        image_points_auto.append(corners_refined)

        # Draw the detected corners
        img_auto = img_train.copy()
        cv2.drawChessboardCorners(img_auto, pattern_size, corners_refined, ret)
        
        cv2.namedWindow("Automatic Corners", cv2.WINDOW_NORMAL)
        cv2.imshow("Automatic Corners", img_auto)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Automatic corner detection failed; invoking manual annotation.")
        manual_corners = get_manual_corners(img_train)
        if manual_corners is not None:
            grid_size_manual = determine_grid_size(manual_corners, horizontal_grid_size=11, vertical_grid_size=8)
            corners_manual_full = interpolate_with_homography(manual_corners, grid_size_manual)

            # Add the refined corners and corresponding object points
            object_points_all.append(objp)
            image_points_all.append(corners_manual_full)

            # Draw the detected corners
            img_with_points = img_train.copy()
            for pt in corners_manual_full:
                x, y = int(pt[0]), int(pt[1])
                cv2.circle(img_with_points, (x, y), 3, (255, 0, 0), -1)
            cv2.namedWindow("Interpolated Points", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("Interpolated Points", 1280, 720)
            cv2.imshow("Interpolated Points", img_with_points)
            cv2.waitKey(0)
            cv2.destroyWindow("Interpolated Points")
        else:
            print("Skipping image since manual annotation is not available.")



Processing image: training_images\WIN_20250211_10_59_26_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_10_59_58_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_00_39_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_01_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_06_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_12_12_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250211_11_17_19_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_40_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_45_Pro.jpg
Automatic corner detection succeeded.

Processing image: training_images\WIN_20250212_12_24_50_Pro.jpg
Automati

Process the final test image

In [60]:
# Load the final test image
final_test_file = "final_test.jpg"
img_final = cv2.imread(final_test_file)
if img_final is None:
    print("Error: Could not load final test image.")
else:
    # Convert the image to grayscale
    gray_final = cv2.cvtColor(img_final, cv2.COLOR_BGR2GRAY)
    
    # Attempt automatic corner detection using your previously defined function
    ret, corners_final = detect_corners_automatically(gray_final, pattern_size)
    
    print("Final Test Image - Detection flag (ret):", ret)
    
    if ret:
        print("Automatic corner detection succeeded on the final test image.")
        # Refine the detected corners for better accuracy
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners_final_refined = cv2.cornerSubPix(gray_final, corners_final, (11, 11), (-1, -1), criteria)

        # Add the refined corners and corresponding object points
        object_points_all.append(objp)
        image_points_all.append(corners_refined)
        
        # Also store these in the automatic lists
        object_points_auto.append(objp)
        image_points_auto.append(corners_refined)
        
        # Draw the detected corners on a copy of the final image
        img_final_drawn = img_final.copy()
        cv2.drawChessboardCorners(img_final_drawn, pattern_size, corners_final_refined, ret)
        
        # Display the final test image with detected corners
        cv2.namedWindow("Final Test Automatic Detection", cv2.WINDOW_NORMAL)
        cv2.imshow("Final Test Automatic Detection", img_final_drawn)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Automatic corner detection failed on the final test image. Please check the image conditions.")

Final Test Image - Detection flag (ret): True
Automatic corner detection succeeded on the final test image.


Camera Calibration Runs

In [61]:
# Determine image size
img_example = cv2.imread(training_files[0])
img_size = (img_example.shape[1], img_example.shape[0])
print("\nImage size (width x height):", img_size)

# Run 1: Use all training images
ret1, cameraMatrix1, distCoeffs1, rvecs1, tvecs1 = cv2.calibrateCamera(
    object_points_all, image_points_all, img_size, None, None)
print("\nRun 1 Calibration Results (All Images):")
print("Camera Matrix:\n", cameraMatrix1)
print("Distortion Coefficients:\n", distCoeffs1)

# Run 2: Use only 10 images with automatic corner detections
if len(object_points_auto) >= 10:
    objpoints_run2 = object_points_auto[:10]
    imgpoints_run2 = image_points_auto[:10]
else:
    print("Not enough automatic images for Run 2; using available automatic images.")
    objpoints_run2 = object_points_auto
    imgpoints_run2 = image_points_auto

ret2, cameraMatrix2, distCoeffs2, rvecs2, tvecs2 = cv2.calibrateCamera(
    objpoints_run2, imgpoints_run2, img_size, None, None)
print("\nRun 2 Calibration Results (10 Automatic Images):")
print("Camera Matrix:\n", cameraMatrix2)
print("Distortion Coefficients:\n", distCoeffs2)

# Run 3: Use only 5 images from the automatic ones
if len(objpoints_run2) >= 5:
    objpoints_run3 = objpoints_run2[:5]
    imgpoints_run3 = imgpoints_run2[:5]
else:
    print("Not enough images for Run 3; using available images from Run 2.")
    objpoints_run3 = objpoints_run2
    imgpoints_run3 = imgpoints_run2

ret3, cameraMatrix3, distCoeffs3, rvecs3, tvecs3 = cv2.calibrateCamera(
    objpoints_run3, imgpoints_run3, img_size, None, None)
print("\nRun 3 Calibration Results (5 Automatic Images):")
print("Camera Matrix:\n", cameraMatrix3)
print("Distortion Coefficients:\n", distCoeffs3)


Image size (width x height): (1920, 1080)

Run 1 Calibration Results (All Images):
Camera Matrix:
 [[1.33606942e+03 0.00000000e+00 9.54817778e+02]
 [0.00000000e+00 1.33469350e+03 5.27633112e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[ 0.04274071 -0.17179898  0.00348209 -0.00901399 -0.03228642]]

Run 2 Calibration Results (10 Automatic Images):
Camera Matrix:
 [[1.31618383e+03 0.00000000e+00 9.89614600e+02]
 [0.00000000e+00 1.31722943e+03 5.52075148e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[-9.34711794e-02  1.04047242e+00  7.20903883e-03  1.44363019e-03
  -2.22530159e+00]]

Run 3 Calibration Results (5 Automatic Images):
Camera Matrix:
 [[1.39305249e+03 0.00000000e+00 1.00025702e+03]
 [0.00000000e+00 1.39116240e+03 5.49803109e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[-2.64399360e-01  3.50304858e+00  1.54716526e-02  5.56941211e-03
  -9.01643131e+00]]


Testing the calibration (will take out when submitting)

In [62]:
# (After calibration runs, for example, after Cell 4)

# Re-read the first training image (or choose any other image that shows the chessboard)
img = cv2.imread(training_files[22])
if img is None:
    print("Error: Could not load the training image for overlay.")
    exit()

# Define 3D points for coordinate axes (length = 3 squares, for example)
axis = np.float32([[3*square_size, 0, 0],
                   [0, 3*square_size, 0],
                   [0, 0, -3*square_size]]).reshape(-1, 3)

# Use the rotation and translation vectors from one of the calibration images.
# Here we use the first set from Run 1 (rvecs1[0] and tvecs1[0]).
# Project the 3D axis points to the image plane using the first calibration image's pose.
imgpts, _ = cv2.projectPoints(axis, rvecs1[22], tvecs1[22], cameraMatrix1, distCoeffs1)

# Convert the reference corner (e.g., the first detected chessboard corner) to integer coordinates.
corner = tuple(map(int, image_points_all[22][0].ravel()))

# Convert the projected axis points to integer coordinates.
x_axis = tuple(map(int, imgpts[0].ravel()))
y_axis = tuple(map(int, imgpts[1].ravel()))
z_axis = tuple(map(int, imgpts[2].ravel()))

# Copy the image for drawing the axes.
img_axes = img.copy()
img_axes = cv2.line(img_axes, corner, x_axis, (255, 0, 0), 5)  # X-axis in blue
img_axes = cv2.line(img_axes, corner, y_axis, (0, 255, 0), 5)  # Y-axis in green
img_axes = cv2.line(img_axes, corner, z_axis, (0, 0, 255), 5)  # Z-axis in red

cv2.namedWindow("3D Axes Overlay", cv2.WINDOW_NORMAL)
cv2.imshow("3D Axes Overlay", img_axes)
cv2.waitKey(0)
cv2.destroyAllWindows()